# Data Merging
Notebook to merge data from provided files

In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd

# Allows the use of display() for DataFrames
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import missingno as msno

from utils import *

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [27]:
import gc
gc.collect()

201

## Ext Bureau

In [3]:
bureau = pd.read_csv('input/bureau.csv.zip')
bureau.head(2)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN


In [4]:
bureau_balance = pd.read_csv('input/bureau_balance.csv.zip')
display(bureau_balance.head(2))

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C


Merging bureau and balance data

In [5]:
bureau_balance_by_id = bureau_balance.groupby('SK_ID_BUREAU')
bureau_grouped_size = bureau_balance_by_id['MONTHS_BALANCE'].size()
bureau_grouped_max = bureau_balance_by_id['MONTHS_BALANCE'].max()
bureau_grouped_min = bureau_balance_by_id['MONTHS_BALANCE'].min()

bureau_counts = bureau_balance_by_id['STATUS'].value_counts(normalize = False)
bureau_counts_unstacked = bureau_counts.unstack('STATUS')
bureau_counts_unstacked.columns = ['STATUS_0', 'STATUS_1','STATUS_2','STATUS_3','STATUS_4','STATUS_5','STATUS_C','STATUS_X',]
bureau_counts_unstacked['MONTHS_COUNT'] = bureau_grouped_size
bureau_counts_unstacked['MONTHS_MIN'] = bureau_grouped_min
bureau_counts_unstacked['MONTHS_MAX'] = bureau_grouped_max

bureau = bureau.join(bureau_counts_unstacked, how='left', on='SK_ID_BUREAU')

#bureau = pd.get_dummies(bureau)

Let's group by sk_id_curr and take average, this way we get something we can merge with app_train/test data

In [6]:
bureau_by_skid = bureau.groupby('SK_ID_CURR')
avg_bureau = bureau_by_skid.mean()
avg_bureau['buro_count'] = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']

In [7]:
avg_bureau = avg_bureau.reset_index()
avg_bureau.head(2)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTHS_COUNT,MONTHS_MIN,MONTHS_MAX,buro_count
0,100001,5896633.000,-735.0,0.0,82.428571,-825.5,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143,4.428571,1.0,NaN,NaN,NaN,NaN,27.500000,6.0,24.571429,-23.571429,0.0,7
1,100002,6153272.125,-874.0,0.0,-349.000000,-697.5,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000,5.625000,4.5,NaN,NaN,NaN,NaN,3.833333,2.5,13.750000,-28.250000,-15.5,8


Previous applications groups by sk id as well and take average, like for bureau data

In [8]:
previous_application = pd.read_csv('input/previous_application.csv.zip')
previous_application.head(2)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0


In [9]:
#previous_application = pd.get_dummies(previous_application)
avg_previous_app = previous_application.groupby('SK_ID_CURR').mean()
cnt_previous_app = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_previous_app['NUM_APPS'] = cnt_previous_app['SK_ID_PREV']

In [10]:
avg_previous_app = avg_previous_app.reset_index()

Credit card balance

In [11]:
POS_CASH_balance = pd.read_csv('input/POS_CASH_balance.csv.zip')
POS_CASH_balance.head(2)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0


processing categorical columns merge

In [12]:
nunique_status = POS_CASH_balance[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
max_status = POS_CASH_balance[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').max()
POS_CASH_balance['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
POS_CASH_balance['MAX_STATUS'] = max_status['NAME_CONTRACT_STATUS']

POS_CASH_balance.drop(columns=['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], inplace=True)

In [13]:
POS_CASH_balance.head(2)

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NUNIQUE_STATUS,MAX_STATUS
0,182943,-31,48.0,45.0,0,0,NaN,NaN
1,367990,-33,36.0,35.0,0,0,NaN,NaN


In [14]:
avg_POS_CASH_balance = POS_CASH_balance.groupby('SK_ID_CURR').mean().reset_index()

In [15]:
avg_POS_CASH_balance.head(2)

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NUNIQUE_STATUS
0,100001,-72.555556,4.0,1.444444,0.777778,0.777778,NaN
1,100002,-10.000000,24.0,15.000000,0.000000,0.000000,NaN


Payment data

In [16]:
installments_payments = pd.read_csv('input/installments_payments.csv.zip')
installments_payments.head(2)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525


Group be skid and take mean, max and min values for each grouping

In [17]:
avg_payments = installments_payments.groupby('SK_ID_CURR').mean()
max_payments = installments_payments.groupby('SK_ID_CURR').max()
min_payments = installments_payments.groupby('SK_ID_CURR').min()

Credit cards data

In [18]:
credit_card_balance = pd.read_csv('input/credit_card_balance.csv.zip')
credit_card_balance.head(2)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.00,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.08,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0


In [19]:
# processing categorical columns merge
nunique_status = credit_card_balance[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
max_status = credit_card_balance[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').max()
credit_card_balance['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
credit_card_balance['MAX_STATUS'] = max_status['NAME_CONTRACT_STATUS']

credit_card_balance.drop(columns=['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], inplace=True)

In [20]:
credit_card_balance.head(2)

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NUNIQUE_STATUS,MAX_STATUS
0,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.00,0.000,0.000,0.0,1,0.0,1.0,35.0,0,0,NaN,NaN
1,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.08,64875.555,64875.555,1.0,1,0.0,0.0,69.0,0,0,NaN,NaN


In [21]:
credit_card_balance = credit_card_balance.groupby('SK_ID_CURR').mean().reset_index()

In [22]:
credit_card_balance.head(2)

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NUNIQUE_STATUS
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.0,0.0,NaN
1,100011,-38.5,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,4520.067568,52402.088919,54433.179122,54433.179122,0.054054,0.054054,0.0,0.0,25.767123,0.0,0.0,1.0


## Merging everything to train and test data

In [24]:
app_train = pd.read_csv('input/application_train.csv.zip')

In [28]:
app_train = app_train.merge(avg_previous_app, how='left', on='SK_ID_CURR')

28

In [29]:
app_train = app_train.merge(avg_POS_CASH_balance, how='left', on='SK_ID_CURR')

38

In [32]:
app_train = app_train.merge(credit_card_balance, how='left', on='SK_ID_CURR')

65

In [33]:
app_train = app_train.merge(avg_payments.reset_index(), how='left', on='SK_ID_CURR')

In [34]:
app_train = app_train.merge(min_payments.reset_index(), how='left', on='SK_ID_CURR')

In [36]:
app_train = app_train.merge(max_payments.reset_index(), how='left', on='SK_ID_CURR')

In [39]:
app_train.shape

(307511, 191)

In [40]:
app_train.to_csv('input/train.csv')

Test data

In [23]:
app_test = pd.read_csv('input/application_test.csv.zip')

In [24]:
app_test = app_test.merge(avg_previous_app, how='left', on='SK_ID_CURR')

In [25]:
app_test = app_test.merge(avg_POS_CASH_balance, how='left', on='SK_ID_CURR')

In [26]:
app_test = app_test.merge(credit_card_balance, how='left', on='SK_ID_CURR')

In [27]:
app_test = app_test.merge(avg_payments.reset_index(), how='left', on='SK_ID_CURR')

In [28]:
app_test = app_test.merge(min_payments.reset_index(), how='left', on='SK_ID_CURR')

In [29]:
app_test = app_test.merge(max_payments.reset_index(), how='left', on='SK_ID_CURR')

In [30]:
app_test.shape

(48744, 190)

In [31]:
app_test.to_csv('input/test.csv')